In [1]:
#@title Deja el CSV con Transcripcion en caser2019/TransAWSResults.csv y los ficheros en Results
import pandas as pd
import os
import requests
import time
from datetime import datetime
import boto3
import json

#bucket_name = "******"
#region = '"******'
#AWS_AUDIO_LOCATOR = ""******"
profile = ""******"
bucket_name = ""******"
region = ""******"
key_id = '"******'
secret_key = '"******'
AWS_AUDIO_LOCATOR = f'https://{bucket_name}.s3.{region}.amazonaws.com/'
os.environ['AWS_DEFAULT_REGION'] = ""******"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = "c:\\users\\diego\\.aws\\credentials\\config"
session = boto3.session.Session(aws_access_key_id=key_id,aws_secret_access_key=secret_key)
s3 = session.resource('s3')
boto3.setup_default_session(aws_access_key_id=key_id,aws_secret_access_key=secret_key)
transcribe = boto3.client('transcribe', region_name=region)
s3_client = boto3.client('s3')
LOCALE = "es-ES"
colsjbs=("Fichero", "Trabajo", "Timestamp")
colsdf=("Fichero", "Trabajo", "Timestamp", "Transcript")
dire="Caser\\"

jobsfilename= "TransAWSJobs.csv"
jobsurl=AWS_AUDIO_LOCATOR + jobsfilename
jbs = pd.read_csv(jobsurl, sep=";", usecols=colsjbs)

resultsfilename= "TransAWSResults.csv"
resultsurl=AWS_AUDIO_LOCATOR + resultsfilename
df = pd.read_csv(resultsurl, sep=";", usecols=colsdf)

transcritos = list(df["Fichero"].values)
print("{} Hechos, {} por transcribir ".format(len(transcritos), len(jbs) - len(transcritos)))
while len(transcritos) < len(jbs):
    for ix, f in jbs.iterrows():
        fname = f["Fichero"]
        job = f["Trabajo"]
        if fname not in transcritos:
            status = transcribe.get_transcription_job(TranscriptionJobName=job)
            if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
                break
            print("{} no esta listo...".format(fname))
    if status['TranscriptionJob']['TranscriptionJobStatus'] == 'COMPLETED':
        print("{} Completed".format(fname))
        res = status['TranscriptionJob']['Transcript']['TranscriptFileUri']
        resjson = requests.get(res).json()
        r = resjson["results"]["transcripts"][0]["transcript"]
        #print(resjson)
        jsonname = '{}.json'.format(fname[:-4].replace("/","-"))
        localname = dire + jsonname
        s3name = "Results/" + jsonname
        with open(localname, 'w', encoding='utf8') as json_file:
            json.dump(resjson, json_file, ensure_ascii=False)
            s3_client.upload_file(localname, bucket_name, s3name)
            s3.ObjectAcl(bucket_name, s3name).put(ACL='public-read')

        df = df.append(pd.DataFrame([[fname, job, str(datetime.now()), r]], columns=colsdf))
        df.to_csv(dire + resultsfilename, sep=";", encoding="utf_8", index=False)
        s3_client.upload_file(dire + resultsfilename, bucket_name, resultsfilename)
        s3.ObjectAcl(bucket_name, resultsfilename).put(ACL='public-read')
        transcritos.append(fname)
        print("{} Hechos, {} por transcribir ".format(len(transcritos), len(jbs) - len(transcritos)))
    elif status['TranscriptionJob']['TranscriptionJobStatus'] == 'FAILED':
        print("{} Failed".format(fname))
    else:
        time.sleep(5)
        print("Trying again...")

print("FINITO!!!!!!!")




24 Hechos, 19 por transcribir 
llamadas_wav_1/SC_1_6515279231372571800_6515279233664757929_8693505_stream0.wav Completed
25 Hechos, 18 por transcribir 
llamadas_wav_1/SC_1_6515279291502113998_6515279293794300083_8693544_stream0.wav Completed
26 Hechos, 17 por transcribir 
llamadas_wav_1/SC_1_6515281529180077693_6515281531472261938_8693792_stream0.wav Completed
27 Hechos, 16 por transcribir 
llamadas_wav_1/SC_1_6515281984446595198_6515281986738795432_8693827_stream0.wav Completed
28 Hechos, 15 por transcribir 
llamadas_wav_1/SC_1_6515282920749466833_6515282923041666218_8693914_stream0.wav Completed
29 Hechos, 14 por transcribir 
llamadas_wav_1/SC_1_6515284350973577877_6515284353265776146_8694153_stream0.wav Completed
30 Hechos, 13 por transcribir 
llamadas_wav_1/SC_1_6515285214262005222_6515285220849170144_8694306_stream0.wav Completed
31 Hechos, 12 por transcribir 
llamadas_wav_1/SC_1_6515285626578866146_6515285628871063376_8694240_stream0.wav Completed
32 Hechos, 11 por transcribir 
l